In [10]:
import mercantile
import requests
from vt2geojson.tools import vt_bytes_to_geojson
import geopandas as gpd
import pandas as pd

from shapely.geometry import shape


from osm2geojson import json2geojson


import logging
from datetime import datetime, timezone
from tqdm import tqdm
import json


In [11]:
### Testing smaller area: Wildau

In [12]:

#{{geocodeArea:"Wildau"}}->.searchArea;
query = """
[out:json][timeout:25];

area["name"="Wildau"][admin_level=8]->.searchArea;

(
  relation["boundary"="administrative"](area.searchArea);
);

out body;
>;
out skel qt;
"""



# Send request
url = "http://overpass-api.de/api/interpreter"
r = requests.get(url, params={'data': query})

# Convert to GeoJSON
res_geojson = json2geojson(r.json())

# Load as GeoDataFrame
admin_boundary = gpd.GeoDataFrame.from_features(res_geojson, crs="EPSG:4326")
admin_boundary

,geometry,type,id,tags
0,"MULTIPOLYGON (((13.56302 52.34968, 13.5617 52....",relation,55775,"{'TMC:cid_58:tabcd_1:Class': 'Area', 'TMC:cid_..."
1,"MULTIPOLYGON (((13.58977 52.32068, 13.59294 52...",relation,55776,"{'TMC:cid_58:tabcd_1:Class': 'Area', 'TMC:cid_..."
2,POINT (13.62933 52.3508),node,59143941,NaN
3,POINT (13.63487 52.32144),node,240111082,NaN


In [15]:
# Get the bounds of the polygon
polygon = admin_boundary[admin_boundary["id"] == 55776].geometry.iloc[0]
minx, miny, maxx, maxy = polygon.bounds

# === Define bounding box (Berlin or any area) ===
bbox = [minx, miny, maxx, maxy]  # must be defined earlier

# === Get intersecting tiles at zoom 14 ===
tiles = list(mercantile.tiles(*bbox, zooms=[14]))
total_tiles = len(tiles)

total_tiles

16

### Brandneburg

In [17]:
#{{geocodeArea:"Brandenburg"}}->.searchArea;
query = """
[out:json][timeout:25];

area["name"="Brandenburg"][admin_level=4]->.searchArea;

(
    relation["boundary"="administrative"][admin_level=4]["ISO3166-2"="DE-BB"](area.searchArea);
);

out body;
>;
out skel qt;
"""

# Send request
url = "http://overpass-api.de/api/interpreter"
r = requests.get(url, params={'data': query})

# Convert to GeoJSON
res_geojson = json2geojson(r.json())

# Load as GeoDataFrame
admin_boundary = gpd.GeoDataFrame.from_features(res_geojson, crs="EPSG:4326")
admin_boundary


,geometry,type,id,tags
0,"MULTIPOLYGON (((11.27106 53.12183, 11.26953 53...",relation,62504,"{'ISO3166-2': 'DE-BB', 'TMC:cid_58:tabcd_1:Cla..."
1,POINT (13.05914 52.40093),node,1695218178,NaN
2,POINT (13.24613 52.84555),node,473862587,NaN


In [18]:
# Get the bounds of the polygon
polygon = admin_boundary[admin_boundary["id"] == 62504].geometry.iloc[0]
minx, miny, maxx, maxy = polygon.bounds

# === Define bounding box (Berlin or any area) ===
bbox = [minx, miny, maxx, maxy]  # must be defined earlier

# === Get intersecting tiles at zoom 14 ===
tiles = list(mercantile.tiles(*bbox, zooms=[14]))
total_tiles = len(tiles)

total_tiles


26565

In [ ]:
## Brandenburg apporx 25k tiles takes ~3h

# Set up logging
logging.basicConfig(
    filename="tile_processing.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# === Set up ===
tile_layer = "sequence"  # or 'image' for points
tile_coverage = "mly1_computed_public"


# Access the token
# Load the JSON config
with open('config.json') as f:
    config = json.load(f)
ACCESS_TOKEN = config.get("ACCESS_TOKEN")

# === Get intersecting tiles at zoom 14 ===
tiles = list(mercantile.tiles(*bbox, zooms=[14]))
total_tiles = len(tiles)
logging.info(f"Total number of tiles to process: {total_tiles}")


# === Collect all features into one GeoDataFrame
gdf_all = gpd.GeoDataFrame(columns=["geometry"], crs="EPSG:4326")

#for i, tile in enumerate(tqdm(tiles[:5], desc="Processing tiles"), start=1):  # or all tiles
for i, tile in enumerate(tqdm(tiles, desc="Processing tiles"), start=1):  # or all tiles
    logging.info(f"Processing tile {i}/{total_tiles}: {tile.z}/{tile.x}/{tile.y}...")
    url = f"https://tiles.mapillary.com/maps/vtp/{tile_coverage}/2/{tile.z}/{tile.x}/{tile.y}?access_token={ACCESS_TOKEN}"
    response = requests.get(url)
    
    if response.status_code != 200:
        logging.warning(f"Error fetching tile {tile.x}/{tile.y}: {response.status_code}")
        continue

    try:
        # Decode tile into GeoJSON format
        geojson = vt_bytes_to_geojson(response.content, tile.x, tile.y, tile.z, layer=tile_layer)
        features = geojson.get("features", [])

        if not features:
            logging.info(f"No features found in tile {tile.x}/{tile.y}.")
            continue

        # Convert to GeoDataFrame
        gdf_tile = gpd.GeoDataFrame.from_features(features, crs="EPSG:4326")

        gdf_tile['captured_at'] = gdf_tile['captured_at'].apply(lambda x: datetime.fromtimestamp(x / 1000, tz=timezone.utc))
        gdf_tile = gdf_tile[gdf_tile['captured_at'] >= datetime(2023, 1, 1, tzinfo=timezone.utc)]
        gdf_tile['captured_at'] = gdf_tile['captured_at'].dt.strftime('%Y-%m-%d')

        if len(gdf_tile) == 0:
            logging.info(f"No features newer than 2023 in tile {tile.x}/{tile.y}.")
            continue
        
        # Append to master dataframe using concat for better performance with many tiles
        gdf_all = gpd.GeoDataFrame(pd.concat([gdf_all, gdf_tile], ignore_index=True), crs="EPSG:4326")

    except Exception as e:
        logging.error(f"Error decoding tile {tile.x}/{tile.y}: {e}")
        break

logging.info(f"✅ Done. Collected {len(gdf_all)} features.")

# Save the collected GeoDataFrame to a Parquet file

# Get current date in YYYY-MM-DD format
current_date = datetime.now().strftime('%Y-%m-%d')

# Add date to filename
filename = f"mapillary_coverage_23_bb_{current_date}.parquet"

# Save with the new filename
gdf_all.to_parquet(filename, index=False)
logging.info(f"Saved data to {filename}")

Processing tiles: 100%|██████████| 26565/26565 [2:35:18<00:00,  2.85it/s]   
